In [34]:
import pandas as pd
import json
import os
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.model_selection import train_test_split


In [2]:
path = r"..\cybernews\data\noise_jsons"
js_ls = []
for file in os.listdir(path):
    with open(f"{path}/{file}", encoding="utf-8") as fp:
        js = json.load(fp)
        for el in js:
            js_ls.append(el)
df = pd.json_normalize(js_ls)
    

In [3]:
oldpath = r"..\cybernews\\data\oldarts.json"
with open(oldpath, encoding="utf-8") as f:
    s = json.load(f)
df2 = pd.json_normalize(s)
df2.head()

,title,author,date,url,source,tags,body,Rating,Filename,Relevant
0,Florida hack highlights security shortages in ...,Sean Lyngaas,2021-02-10 00:00:00,https://www.cyberscoop.com/florida-water-hack-...,CyberScoop,"[critical infrastructure, Dragos, industrial c...",A hack that apparently affected a Florida wate...,2,Cyber_Clips_February_11_2021.docx,1
1,We Must Reorient US Cyber Strategy Around the ...,Dmitri Alperovitch,2021-02-10 00:00:00,https://www.defenseone.com/ideas/2021/02/five-...,Defense One,"[Commentary, Cyber, Congress, China, Russia, I...",This oped is adapted from Dmitri Alperovitch's...,4,Cyber_Clips_February_11_2021.docx,1
2,DHS announces new measures to boost nation's c...,Maggie Miller,2021-02-22 17:32:00,https://thehill.com/policy/cybersecurity/53994...,The Hill,"[DHS, CISA, Alejandro Mayorkas, Bennie Thompso...",The Department of Homeland Security (DHS) on M...,4,Cyber_Clips_February_23_2021.docx,1
3,"Biden calls for creating 'rules' on cyber, tec...",Maggie Miller,2021-02-19 13:18:00,https://thehill.com/policy/cybersecurity/53959...,The Hill,"[China, Russia, Biden, Vladimir Putin, Joe Bid...",President Biden on Friday called on the United...,2,Cyber_Clips_February_22_2021.docx,1
4,Ryuk ransomware develops worm-like capabilitie...,Shannon Vavra,2021-03-03 00:00:00,https://www.cyberscoop.com/ryuk-ransomware-dev...,CyberScoop,"[France, health care, ransomware, Ryuk ransomw...",A new sample of Ryuk ransomware appears to hav...,6,Cyber_Briefing_March_04_2021.docx,1


In [4]:
df2.body[0]

'A hack that apparently affected a Florida water facility’s chemical setting is emblematic of a water sector that’s short on money, cybersecurity personnel and often reliant on the practices of vendors, experts say.\nThe Feb. 5 incident in Oldsmar, a Florida town of 15,000 people, involved a still-unidentified hacker infiltrating the local water treatment facility’s computer system and trying to increase the amount of sodium hydroxide to a potentially dangerous level, local authorities said. The substance is used in the water purification process but can be toxic at higher levels. No harm was done to public health — the facility had safety checks in place — but the level of access obtained by the attacker has prompted calls for tighter security in the sector.\nThe breach is an uncomfortable reminder that water facilities struggle to invest as much money in effective security as other industrial organizations, even as they face “an increase in the frequency, diversity, and complexity of

In [5]:
merged=pd.concat([df, df2], ignore_index=True)

In [6]:
merged.head()

,title,author,date,url,source,tags,body,Relevant,Rating,Filename
0,"In a cyberattack disaster, DoD needs backup sq...",Mark Pomerleau,2021-03-16 08:00:00,https://www.c4isrnet.com/cyber/2021/03/16/in-a...,C4ISRNET,None,WASHINGTON — With a growing number of cyber br...,0,NaN,NaN
1,Cyber Command task force focuses on emerging t...,Mark Pomerleau,2021-03-08 21:13:00,https://www.c4isrnet.com/cyber/2021/03/08/cybe...,C4ISRNET,None,WASHINGTON — A task force with the Pentagon’s ...,0,NaN,NaN
2,Cyber Command works to address criticism over ...,Mark Pomerleau,2021-03-12 16:53:00,https://www.c4isrnet.com/cyber/2021/03/12/cybe...,C4ISRNET,None,WASHINGTON — U.S. Cyber Command is working to ...,0,NaN,NaN
3,More work needed to integrate cyber and inform...,Mark Pomerleau,2021-03-06 14:41:00,https://www.c4isrnet.com/information-warfare/2...,C4ISRNET,None,WASHINGTON — The Department of Defense needs t...,0,NaN,NaN
4,"After SolarWinds, US needs to toughen cyber de...",Joe Gould,2021-02-23 13:05:00,https://www.c4isrnet.com/2021/02/23/after-sola...,C4ISRNET,None,WASHINGTON ― In the wake of a sweeping hack th...,0,NaN,NaN


In [26]:
stops = stopwords.words('english')
analyze = CV(stop_words=stops, min_df = 15, max_df=.5)
ps = PorterStemmer()
def stop_removal(text : str): 
    toks=analyze(text)
    return ' '.join([ps.stem(word) for word in toks if not ps.stem(word) in stops])

In [27]:
mat = analyze.fit_transform(merged["body"])

In [28]:
# analyze.get_feature_names()

In [29]:
counts = pd.DataFrame(mat.toarray(), columns=analyze.get_feature_names())
counts

,000,10,100,11,12,15,16,18,19,20,...,write,writing,written,wrote,year,years,yet,york,young,zero
0,1,2,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,3,1,2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
450,2,0,1,0,0,0,0,0,0,0,...,0,0,0,1,3,0,0,0,0,0
451,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,2,3,0,0,0,0
452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [30]:
# counts["Relevant_"] = merged.Relevant

In [31]:
X_train, X_test, y_train, y_test = train_test_split(counts, merged.Relevant, test_size=.1, random_state=109, stratify=merged.Relevant)

In [32]:
y_test

237    0
141    0
268    0
394    1
316    0
319    0
386    0
115    0
166    0
297    0
309    0
51     0
272    0
197    0
228    0
308    0
64     0
287    0
75     0
65     0
403    1
102    0
133    0
245    0
442    1
122    0
7      0
173    0
407    1
15     0
262    0
267    0
424    1
298    0
289    0
304    0
422    1
318    0
260    0
157    0
258    0
385    0
400    1
180    0
186    0
238    0
Name: Relevant, dtype: int64

In [36]:
model=LogisticRegression(random_state=109, max_iter=100).fit(X_train, y_train)

c:\users\tlebr\envs\fdd\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
merged.groupby("Relevant").count()

,title,author,date,url,source,tags,body,Rating,Filename
Relevant,,,,,,,,,
0,388,388,388,388,388,154,388,0,0
1,66,66,66,66,66,25,66,66,66


In [38]:
model.score(X_train, y_train)

0.9877450980392157

In [45]:
model.predict(X_test)

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0], dtype=int64)

tokens